In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # or "3" for GPU 3


In [ ]:
# DSF_Feature_Extraction.ipynb (Notebook for DSF)

import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime
from transformers import TFViTForImageClassification, ViTImageProcessor

print(f"\U0001F680 Starting DSF feature extraction at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

image_dir = "/home/shahid/000.Damage Severity/aman/"
model_name = "microsoft/swin-tiny-patch4-window7-224"
model_save_path = "./swin_transformer_dsf"

df_train = pd.read_table(os.path.join(image_dir, "embeddings/task_damage_text_img_train.tsv"))
df_val   = pd.read_table(os.path.join(image_dir, "embeddings/task_damage_text_img_dev.tsv"))
df_test  = pd.read_table(os.path.join(image_dir, "embeddings/task_damage_text_img_test.tsv"))
label_mapping = {"severe_damage": 0, "mild_damage": 1, "little_or_no_damage": 2}
for df in [df_train, df_val, df_test]:
    df["damage_label"] = df["label"].map(label_mapping)

processor = ViTImageProcessor.from_pretrained(model_name)
model = TFViTForImageClassification.from_pretrained(model_name, num_labels=3, ignore_mismatched_sizes=True)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

if not os.path.exists(model_save_path):
    model.save(model_save_path)

def load_image(image_path):
    full_path = os.path.join(image_dir, image_path)
    if not os.path.exists(full_path):
        return None, None
    img = cv2.imread(full_path)
    if img is None:
        return None, None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (224, 224))
    inputs = processor(images=img_resized, return_tensors="tf")
    return img_resized, inputs["pixel_values"]

# === DSF computation ===
def compute_dsf(image, img_tensor):
    img_tensor = tf.convert_to_tensor(img_tensor, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(img_tensor)
        preds = model(img_tensor)
        severe_score = preds.logits[0, 0]  # Score for Severe class
    grads = tape.gradient(severe_score, img_tensor)
    saliency = tf.reduce_max(tf.abs(grads), axis=-1)
    saliency = tf.squeeze(saliency, axis=0)
    saliency = tf.image.resize(saliency[..., tf.newaxis], (224, 224))[..., 0].numpy()
    features = [np.mean(saliency), np.max(saliency), np.std(saliency)]
    return np.nan_to_num(features), saliency

def extract_dsf_features(df, split_name):
    features = []
    predictions = []
    image_paths = []
    actual_labels = []
    for idx, row in df.iterrows():
        print(f"[{idx + 1}/{len(df)}] Processing: {row['image']}")
        img, img_tensor = load_image(row['image'])
        if img is None or img_tensor is None:
            continue
        dsf_feat, _ = compute_dsf(img, img_tensor)
        pred_label = tf.argmax(model(img_tensor).logits, axis=1).numpy()[0]
        features.append(dsf_feat)
        predictions.append(pred_label)
        image_paths.append(row['image'])
        actual_labels.append(row['damage_label'])
        print(f"    ✅ Feature extracted | Mean: {dsf_feat[0]:.4f} | Pred: {pred_label} | Actual: {row['damage_label']}")
    features = np.array(features)
    np.save(f"{split_name}_dsf_features.npy", features)
    print(f"Saved {split_name} DSF features: {features.shape} → {split_name}_dsf_features.npy")

    df_out = pd.DataFrame({
        'image': image_paths,
        'actual_label': actual_labels,
        'predicted_label': predictions
    })
    df_out.to_csv(f"{split_name}_dsf_predictions.csv", index=False)
    print(f"Saved predictions to {split_name}_dsf_predictions.csv")
    return features

train_features = extract_dsf_features(df_train, "train")
val_features   = extract_dsf_features(df_val, "val")
test_features  = extract_dsf_features(df_test, "test")

print("\n✅ DSF feature extraction complete!")


/home/shahid/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2025-07-02 17:10:17.358309: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-02 17:10:17.397175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-02 17:10:17.471986: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-02 17:10:17.487582: E external/local_xla/xla/stream_exe

🚀 Starting DSF feature extraction at 2025-07-02 17:10:41


You are using a model of type swin to instantiate a model of type vit. This is not supported for all configurations of models and can yield errors.
2025-07-02 17:10:42.583442: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-07-02 17:10:45.473481: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Input to reshape is a tensor with 768000 values, but the requested shape has 2304
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['swin.encoder.layers.2.blocks.3.layernorm_before.weight', 'swin.encoder.layers.3.blocks.0.attention.output.dense.weight', 

INFO:tensorflow:Assets written to: ./swin_transformer_dsf/assets


INFO:tensorflow:Assets written to: ./swin_transformer_dsf/assets


[1/2468] Processing: data_image/hurricane_harvey/8_9_2017/905960092822003712_0.jpg


2025-07-02 17:12:20.470990: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT32 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


    ✅ Feature extracted | Mean: 0.0004 | Pred: 1 | Actual: 0
[2/2468] Processing: data_image/california_wildfires/11_10_2017/918008272363368448_0.jpg
    ✅ Feature extracted | Mean: 0.0000 | Pred: 2 | Actual: 0
[3/2468] Processing: data_image/hurricane_irma/17_9_2017/909396901254090752_0.jpg
    ✅ Feature extracted | Mean: 0.0003 | Pred: 0 | Actual: 0
[4/2468] Processing: data_image/hurricane_maria/24_9_2017/912097936200355841_0.jpg
    ✅ Feature extracted | Mean: 0.0001 | Pred: 1 | Actual: 0
[5/2468] Processing: data_image/hurricane_maria/23_10_2017/922610290281402368_1.jpg
    ✅ Feature extracted | Mean: 0.0004 | Pred: 2 | Actual: 2
[6/2468] Processing: data_image/california_wildfires/15_10_2017/919585500914208768_0.jpg
    ✅ Feature extracted | Mean: 0.0001 | Pred: 0 | Actual: 0
[7/2468] Processing: data_image/hurricane_maria/1_10_2017/914372204787355648_0.jpg
    ✅ Feature extracted | Mean: 0.0003 | Pred: 1 | Actual: 2
[8/2468] Processing: data_image/california_wildfires/21_10_2017